In [ ]:
import os
import imageio 
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import atlite
import matplotlib as mpl
import numpy as np

from PIL import Image
#mpl.use('Agg')

In [ ]:
# %% atlite
cutout = atlite.Cutout('global', x=slice(-180, 180), y=slice(-180, 180),
                       time='2018-05-05', module='era5')
cutout.prepare('wind')

projection = ccrs.Orthographic(0, 30)

In [ ]:
imname = 0
where = range(0,24*5,2)
for t in where:
    fig, ax = plt.subplots(figsize=(5,5), subplot_kw={"projection": projection})
    cutout.data.wnd100m.isel(time=t).plot(ax=ax,
                                          transform=ccrs.PlateCarree(),
                                          add_colorbar=False,
                                          norm=mpl.colors.Normalize(vmin=0, vmax=20),
                                          alpha=.3,
                                          cmap='Blues',
                                          zorder=5)
    ax.coastlines(zorder=6, linewidth=0.5)
    ax.set_title('')
    ax.spines['geo'].set_visible(False)
    print(str(t) + '/' + str(len(where)))
    fig.savefig(f'../atlite-{imname}.png', bbox_inches='tight')
    imname+=1

In [ ]:
def gen_frame(path):
    im = Image.open(path)
    alpha = im.getchannel('A')

    # Convert the image into P mode but only use 255 colors in the palette out of 256
    im = im.convert('RGB').convert('P', palette=Image.ADAPTIVE, colors=255)

    # Set all pixel values below 128 to 255 , and the rest to 0
    mask = Image.eval(alpha, lambda a: 255 if a <=128 else 0)

    # Paste the color of index 255 and use alpha as a mask
    im.paste(255, mask)

    # The transparency index is 255
    im.info['transparency'] = 255
    return im


im1 = gen_frame('../atlite-0.png')
append_images = np.append([f"../atlite-{i}.png" for i in range(0,24)],
                          [f"../atlite-{i}.png" for i in range(23,-1,-1)])
im1.save('../gifs/atlite.gif', save_all=True, append_images=[gen_frame(append_images[i]) for i in range(len(append_images))],
         disposal=2, loop=2, duration=100)
im1.save('../atlite.png', save_all=False, append_images=[])

In [ ]:
# Build GIF
if 0:
    with imageio.get_writer(f'img/martha/atlitegif.gif', mode='I') as writer:
        #komplett hoch
        for filename in [f"img/martha/atlite-{i}.png" for i in range(0,24)]:
            image = imageio.imread(filename)
            writer.append_data(image)
        
        #komplett runter
        for filename in [f"img/martha/atlite-{i}.png" for i in range(23, -1, -1)]:
            image = imageio.imread(filename)
            writer.append_data(image)

In [ ]:
# delete images...
for file in [f"img/martha/atlite-{i}.png" for i in range(1,24)]:
    os.remove(file) 